# bit.ly/symulator-kod
# bit.ly/symulator-model

In [1]:
from flask import Flask
import eventlet.wsgi
import eventlet
import socketio
import base64
from io import BytesIO
from PIL import Image
import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [6]:
model = load_model(r"C:\Users\Marta\Desktop\ML\CHALLENGE 3 Self-driving car\beta_simulator_windows\myModel.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [7]:
sio = socketio.Server()
app = Flask(__name__)

def send_control(steering_angle, throttle):
    sio.emit("steer", data={'steering_angle': str(steering_angle),
                            'throttle': str(throttle) }, skip_sid=True)

In [ ]:
def process_image(img):
        return img[10:130:2, ::4, :]


@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        speed = float(data["speed"])
        image_str = data["image"]

        decoded = base64.b64decode(image_str)
        image = Image.open(BytesIO(decoded))
        image_array = np.asarray(image)
        
        #print(image_array.shape)
        #plt.imshow(image_array)
        #plt.show();        
        #print(data.keys())
        
        # model input 60x80x3
        img = process_image(image_array)
        img_batch = np.expand_dims(img, axis=0)
        steering_angle = float(model.predict(img_batch))
        
        
        # kąt
        # steering_angle = 0.0  # przyjmuje wartości od -1.0 (w lewo) do 1.0 (w prawo)
        # gaz
        throttle = 0.1 # przyjmuje wartości z przedziału <0.0, 1.0>

        if speed < 10:
            throttle = 0.6
            
        if speed > 15:
            throttle = -0.1
        
        send_control(steering_angle, throttle)
    else:
        sio.emit('manual', data={}, skip_sid=True)
        
app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(11160) wsgi starting up on http://0.0.0.0:4567
(11160) accepted ('127.0.0.1', 54287)
127.0.0.1 - - [21/May/2019 15:15:03] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 39.981754
(11160) accepted ('127.0.0.1', 54297)
127.0.0.1 - - [21/May/2019 15:16:07] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 44.774713
(11160) accepted ('127.0.0.1', 54336)
127.0.0.1 - - [21/May/2019 15:21:04] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 16.615741
